A form $(a, b, c)$ is ambiguous if it is equivalent to $(a, -b, c)$, i.e., the class has order $2$ or $1$. For a form to be reduced and ambiguous, its coefficients must satisfy one of three special conditions, $b = 0$, $b = a$ or $a = c$. There is a relationship between the reduced ambiguous forms and the factoriation of the discriminant $d$.

*   Forms of type $(a, 0, c)$: These forms correspond directly to the ways that $-d/4$ can be written as a product of two coprime integers $a$ and $c$.
*   Forms of type $(a, a, c)$ and $(a, b, a)$: These forms are related to factorisations of $-d$. The coefficients $a$ or $b$ appear as factors of $d$.

Let $k$ be the number of distinct prime factors of the discriminant $d$. Then the number of reduced ambiguous forms is $2^{k-1}$.

We wish to find a non-trivial factor of a given composite integer $N$ given the theory of binary quadratic forms. We take a discriminant $d = -kN$, with $k$ a small positive integer, and attempt to construct ambiguous forms of discriminant $d$. We repeatedly pick a form at random and raise it to a suitable power in the class group.

To perform this method we require knowledge of the class number $h(d)$. Our existing method works by exhaustive search by enumerating all possible reduced forms for a given discriminant $d$ by iterating through all possible coefficients $a$ an $b$, of which there are proportionally $\sqrt{-d}$ of each, leading to a time complexity of $O(|d|)$. The magnitude of the discriminant $|d|$ is on the same order as the number we want to factor $N$, so the time complexity is $O(N)$. This is worse than trial division which is approximately $O(\sqrt{N})$.

Instead we fix a positive integer $B$ and assume that $h(d)$ is a product of prime powers less than $B$. We choose a form of discriminant $d$ at random by choosing a small value of $a$, and then solving for $b$ and $c$ if possible. Then we successively raise it to each odd prime power less than $B$. We then repeatedly square (efficiently) this form in the hope of finding an ambiguous form that factors $N$. If this method repeatedly fails we might increase the value of $B$ or change the value of $k$.


In [60]:
import math

def reduce_form(a, b, c):
    '''
    Reduces a quadratic form (a,b,c) to its unique equivalent reduced form.
    '''
    while True:
        # Check if already reduced
        if -a < b <= a and a < c: break
        if 0 <= b <= a and a == c: break

        # Apply S transformation if a > c or a=c and b < 0
        if a > c or (a == c and b < 0):
            a, b, c = c, -b, a
            continue

        # Apply T^n or T^-n transformation to bring b into [-a, a]
        if b > a:
            n = (b + a) // (2 * a)
            b, c = b - 2*a*n, a*n*n - b*n + c
        elif b <= -a:
            n = (-b + a) // (2 * a)
            b, c = b + 2*a*n, a*n*n + b*n + c

    return (a, b, c)

def compose_forms(f1, f2, d):
    '''
    Computes the composition of two primitive forms with discriminant d.
    '''
    a1, b1, c1 = f1
    a2, b2, c2 = f2

    beta = (b1 + b2) // 2
    gamma = (b2 - b1) // 2

    # Use Extended Euclidean Algorithm to solve a1*x + beta*y = m
    m, x, _ = math.gcd(a1, beta), *extended_gcd(a1, beta)[1:]

    n = math.gcd(m, a2)

    # Solve the congruence (m/n)z = (gamma*x - c1*y) (mod a2/n)
    A = m // n
    B = gamma * x - c1 * _
    M = a2 // n
    z = (B * pow(A, -1, M)) % M

    a3 = (a1 * a2) // (n**2)
    b3 = b1 + (2 * a1 * z) // n
    c3 = (b3**2 - d) // (4 * a3)

    return reduce_form(a3, b3, c3)

def extended_gcd(a, b):
    if a == 0: return b, 0, 1
    g, y, x = extended_gcd(b % a, a)
    return g, x - (b // a) * y, y

In [61]:
def power(form, exp, d):
    '''
    Computes form^exp in the class group using exponentiation by squaring.
    '''
    # Determine the identity element
    if d % 4 == 0:
        identity = (1, 0, -d // 4)
    else:
        identity = (1, 1, (1 - d) // 4)

    result = identity
    base = form

    while exp > 0:
        if exp % 2 == 1:
            result = compose_forms(result, base, d)
        base = compose_forms(base, base, d)
        exp //= 2

    return result

def generate_primes(limit):
    '''
    Generate primes up to a limit using a sieve.
    '''
    primes = []
    is_prime = [True] * (limit + 1)
    is_prime[0] = is_prime[1] = False
    for p in range(2, limit + 1):
        if is_prime[p]:
            primes.append(p)
            for i in range(p * p, limit + 1, p):
                is_prime[i] = False
    return primes

def find_starting_form(d, N):
    '''
    Finds a suitable primitive form (a,b,c) for discriminant d.
    '''
    for a in generate_primes(100): # Try small prime 'a' values
        if N % a == 0: continue # 'a' should not be a factor of N

        # We need b^2 = d + 4ac, so b^2 = d (mod 4a)
        for b in range(1, 100):
            if (b*b - d) % (4*a) == 0:
                c = (b*b - d) // (4*a)
                # Ensure it's primitive
                if math.gcd(math.gcd(a,b),c) == 1:
                    return reduce_form(a, b, c)
    return None

def factor_with_forms(N, k, B):
    '''
    Attempts to factor N using the quadratic forms method with parameters k and B.
    '''
    d = -k * N
    if d % 4 not in (0, 1):
        # Invalid discriminant
        return None

    print(f"\n--- Factor N = {N} with k = {k}, B = {B}, d = {d} ---")

    # Find a starting form
    f_start = find_starting_form(d, N)
    if not f_start:
        print("  Could not find a suitable starting form.")
        return None
    print(f"Starting with form: f_start = {f_start}")

    # Raise to the power of all odd prime powers < B
    primes = generate_primes(B)
    E = 1
    for p in primes:
        if p == 2: continue
        p_power = p
        while p_power <= B / p:
            p_power *= p
        E *= p_power

    f_E = power(f_start, E, d)
    print(f"After raising to prime powers < {B}, form is: f_E = {f_E}")

    # Repeatedly square to find an ambiguous form
    g = f_E
    for i in range(1, 100): # Limit squaring to prevent infinite loops
        g = compose_forms(g, g, d)
        # Check for ambiguity
        a, b, c = g
        if b == 0 or b == a or a == c:
            # Ambiguous form found

            # Find factor using GCD
            factor = math.gcd(N, a)
            if 1 < factor < N:
                print(f"\nFound non-trivial factor: {factor}")
                return factor
            else:
                # Ambiguous form gave trivial factor
                pass

    print("Failed to find a factor after extensive squaring.")
    return None

numbers_to_factor = [12597203, 33377419, 49047121]
B_LIMIT = 50
K_LIMIT = 200

for N in numbers_to_factor:
    print(f"=" * 30)
    print(f"Processing N = {N}")
    print(f"=" * 30)
    found_factor = False
    for k_val in range(1, K_LIMIT + 1):
        factor = factor_with_forms(N, k_val, B_LIMIT)
        if factor:
            print(f"Factorisation of {N} is {factor} * {N // factor}\n")
            found_factor = True
            break
    if not found_factor:
        print(f"\nNo factor found for N={N} with k up to {K_LIMIT}.\n")

Processing N = 12597203

--- Factor N = 12597203 with k = 1, B = 50, d = -12597203 ---
Starting with form: f_start = (3, 1, 1049767)
After raising to prime powers < 50, form is: f_E = (699, 349, 4549)
Failed to find a factor after extensive squaring.

--- Factor N = 12597203 with k = 4, B = 50, d = -50388812 ---
Starting with form: f_start = (3, 2, 4199068)
After raising to prime powers < 50, form is: f_E = (2796, -2098, 4899)

Found non-trivial factor: 2153
Factorisation of 12597203 is 2153 * 5851

Processing N = 33377419

--- Factor N = 33377419 with k = 1, B = 50, d = -33377419 ---
Starting with form: f_start = (5, 1, 1668871)
After raising to prime powers < 50, form is: f_E = (325, 9, 25675)
Failed to find a factor after extensive squaring.

--- Factor N = 33377419 with k = 4, B = 50, d = -133509676 ---
Starting with form: f_start = (5, 2, 6675484)
After raising to prime powers < 50, form is: f_E = (1300, -1282, 25991)
Failed to find a factor after extensive squaring.

--- Factor N

The fundamental operations of the entire process is the composition of two forms followed by a reduction. The inputs to this operation are the coefficients $(a, b, c)$ of reduced forms whose sizes are bounded by the discriminant $d = -kN$. Specifically, $a$ can be as large as $O(\sqrt{|d|}) = O(\sqrt{N})$.

The composition operation relies on the extended Euclidean algorithm. The complexity on numbers of size $X$ is polynomial in the number of bits, roughly $O(\log^2 X)$. Since our coefficients are up to $O(\sqrt{N})$, this is $O(\log N)$. Therefore, the time complexity of a single composition followed by a reduction is $O(\log^2 N)$.

The most computationally expensive part of the algorithm is raising the initial form $f$ to the large exponent $E$. The number of compositions it must perform is proportional to the number of bits in the exponent $O(\log E)$. The exponent $E$ is the product of all odd prime powers less than $B$, so $\log E$ is the sum of the logarithms of these prime powers. The prime number theorem shows that $O(\log E) = O(B)$.

Combining these, the total time for this main step is $O(B \log^2 N)$.

*   If we choose a small $B$, the exponentiation step is fast. However, the chance that the class number $h(d)$ is $B$-smooth is low. This means we might have to try many values of $k$ or perform many squarings, making the algorithm run for a long time.
*   If we choose a large $B$, the chance of $h(d)$ being B-smooth is much higher, making the algorithm more likely to succeed quickly. However, the exponentiation step now becomes very slow.

The optimal strategy is to choose a value for $B$ that balances these two competing factors. When this optimisation is done, the resulting complexity is sub-exponential. It is expressed using $L$-notation as
\begin{equation}
    L_N[1/2, c] = \exp(c \sqrt{\log N \cdot \log\log N})
\end{equation}
where $c$ is a small constant.

*   Trial division has a complexity of $O(\sqrt{N})$ which in L-notation is $L_N[1, 1/2]$, hence is exponential.
*   General number field sieve is the fastest known classical factoring algorithm with complexity is $L_N[1/3, c]$. It is asymptotically faster than the class group method because the term in the exponent is $(\log N)^{1/3}$ instead of $(\log N)^{1/2}$.
*   Shor's algorithm in quantum computing has a complexity of $O(\log^3 N)$ which is of polynomial time and is therefore theoretically much faster than any of the classical algorithms.